In [3]:
from typing import Any

In [4]:
# when a type checker sees this:
def double(x):
    return x * 2

# it assumes this:
def double(x: Any):
    return x * 2

In [5]:
# object doesn't support multiply, hence type error.
# more general types support fewer operations
def double(x: object) -> object:
    return 2 * x

> "If possible, avoid creating functions that return Union types, as they put an extra burden on the user -- forcing them to check the type of the returned value at runtime to know what to do with it."

In [7]:
from typing import NamedTuple, Tuple

# Empfohlene Implementierung:
class Coordinate(NamedTuple):
    lat: float
    lon: float

def geohash(coord: Coordinate):
    pass

# suboptimale Alternative, da Reihenfolge von lat/lon kritisch
def geohash(coord: Tuple[float, float]):
    pass

# suboptimale Alternative, da Reihenfolge von lat/lon kritisch
Coordinate = Tuple[float, float]

def geohash(coord: Coordinate):
    pass

# ggf. tricksen mit arg name
def geohash(lat_lon: Tuple[float, float]):
    pass

In [8]:
# Tuple mit beliebiger Anzahl Elementen von Typ "int" möglich
def sum_all_elements(tpl: Tuple[int, ...]):
    pass

## Type Alias

In [ ]:
# python <3.10
Coordinate = Tuple[float, float]

# python >=3.10
from typing import TypeAlias
Coordinate: TypeAlias = tuple[float, float]

## Parametrized Generics


In [ ]:
# falsch. warum?
from random import shuffle
from typing import List, Sequence

def sample(population: Sequence[Any], size: int) -> List[Any]:
    result = list(population)
    shuffle(result)
    return result[:size]

In [ ]:
# korrekt
from random import shuffle
from typing import List, Sequence, TypeVar

T = TypeVar('T') # can be anything

def sample(population: Sequence[T], size: int) -> List[T]:
    result = list(population)
    shuffle(result)
    return result[:size]


# -> Wenn ich eine Sequenz von Typ "int" reinstecke, kommt eine Liste von "int" raus.
#                                  "str"                                  "str" 
#                                   ...                                    ...

In [ ]:
# Sequence vs Iterable
# Für Parameter: Iterable, weil gröber
# Für Return: Sequence, weil genauer
# TODO 
# z.B. kriege ich die Länge jeder Sequence mit len(), aber es gibt Iterables ohne len()!

In [11]:
from collections import Counter
from collections.abc import Iterable

# Version 1
def mode(data: Iterable[int]) -> int:
    """Return the most common data point from a series"""
    pairs = Counter(data).most_common()
    return pairs[0][0]

mode([1, 1, 2, 3, 3, 3, 3, 4])

3

In [13]:
# Version 2
# Typing mit Hinwweisen, dass wir:
#   (1) verschiedene Datentypen reinwerfen können, nicht nur ints
#   (2) immer den gleichen Datentyp rauskriegen, den wir reingesteckt haben
from typing import TypeVar

T = TypeVar('T') 

def mode(data: Iterable[T]) -> T:
    """Return the most common data point from a series"""
    pairs = Counter(data).most_common()
    return pairs[0][0]

mode([1, 1, 2, 3, 3, 3, 3, 4])
mode(['a', 'a', 'b', 'c', 'c', 'c', 'c', 'd'])

'c'

In [18]:
# Das Problem ist, dass die Funktion unhashable types, z.B. lists und dicts, nicht handlen kann:
mode([['a'], ['b']])

TypeError: unhashable type: 'list'

In [ ]:
# also müssen wir das typing Einschränken
# z.B. mit numerischen Datentypen:
from typing import TypeVar, Decimal, Fraction

# "Restricted Type Vars" := TypeVars beschränkt auf bestimmte Datentypen
NumberT = TypeVar('NumberT', float, Decimal, Fraction)

def mode(data: Iterable[NumberT]) -> NumberT:
    pass 

In [ ]:
# aber irgendwie blöd, weil vllt wollen wir mal strings reinstecken
# dafür gibt es "BOUNDED TYPEVARS"
from typing import Hashable

# Bounded TypeVars := TypeVars beschränkt auf alle Typen, die "consistent-with" dem angegeben Typ (hier: Hashable) sind.
HashableT = TypeVar('HashableT', bound=Hashable)

def mode(data: Iterable[HashableT]) -> HashableT:
    pass

In [23]:
# custom types that support specific operators
from typing import Protocol, Any

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

LT = TypeVar('LT', bound=SupportsLessThan)

# supportet alle Objekte, die __lt__() implementiert haben
def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [24]:
# z.B. ints
top([3, 10, 5], 2)

[10, 5]

In [27]:
# oder strings
top(['a', 'd', 'x'], 1)

['x']

In [29]:
# aber eben auch user-defined classes, die __lt__() implementieren!
from dataclasses import dataclass

@dataclass
class Employee:
    name: str
    salary: int
    def __lt__(self, other): return self.salary < other.salary


nils = Employee('nils', 150_000) # AT
mein_chef = Employee('chef', 80_000)
ceo = Employee('CEO', 115_000)

top([nils, mein_chef, ceo], 1)

[Employee(name='nils', salary=150000)]